In [2]:
import pandas as pd
import numpy as np


In [3]:
df=pd.read_csv('reviews.csv')

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


Interested only in reviewtext and overall-these 2 columns-cols of interest


In [5]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-2jvlakrh
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-2jvlakrh
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-cp36-none-any.whl size=11742 sha256=4aa0f921f5e6d92b5479a61bed10a78bc0e1ddd226f8e36899fc02c5e6dbd267
  Stored in directory: /tmp/pip-ephem-wheel-cache-_d8bhi50/wheels/a8/18/22/90afa4bd43247fb9a75b710a4a3fcd94966c022ce9e3c7d0a6
Successfully built preprocess-kgptalkie


In [9]:
import preprocess_kgptalkie as ps
import re

In [10]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)  #remove multiple repeated characters
    return x

#func to pre-process our data

In [11]:
#pre-process the reviewText column row-by-row

df['reviewText']=df['reviewText'].apply(lambda x:get_clean(x))

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [31]:
tfidfvect = TfidfVectorizer(max_features=20000,ngram_range=(1,5),analyzer='char')

In [32]:
x=tfidfvect.fit_transform(df['reviewText'])
y=df['overall']

#reviewText is the only feature using which star rating must be predicted

In [33]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [42]:
svc = LinearSVC(C=30,class_weight='balanced')

#C is the optimizer-to get better accuracy %,use this

In [43]:
svc.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=30, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [44]:
predictions=svc.predict(x_test)

In [45]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         1.0       0.36      0.23      0.28        39
         2.0       0.19      0.11      0.14        55
         3.0       0.25      0.28      0.26       134
         4.0       0.34      0.34      0.34       451
         5.0       0.77      0.78      0.78      1374

    accuracy                           0.62      2053
   macro avg       0.38      0.35      0.36      2053
weighted avg       0.62      0.62      0.62      2053



In [53]:
def displayPredictedRating(predicted_rating):

  if(predicted_rating[0] == 1.0):
    print("1 star")
  elif(predicted_rating[0] == 2.0):
    print("2 star")
  elif(predicted_rating[0] == 3.0):
    print("3 star")
  elif(predicted_rating[0] == 4.0):
    print("4 star")
  else:
    print("5 star")

In [58]:
user_review=input("Enter your review on our product: ")
clean_data=get_clean(user_review)

vect=tfidfvect.transform([clean_data])

predicted_rating=svc.predict(vect)

displayPredictedRating(predicted_rating)



Enter your review on our product: Good but the quality does not meet the standards.So,you must improve a lot.
3 star
